In [ ]:
import pandas as pd

quantlcms = pd.read_csv("QuantitativeLCMS.csv", index_col=0)

quantlcms

In [ ]:
atlas = quantlcms[['Male',	'Male.1',	'Male.2',	'Male.3']] ### mean across males to augment our atlas

In [ ]:
ref = atlas.iloc[1:,:].astype(float).mean(axis=1)
ref

In [ ]:
annots = pd.read_csv("goslin_output.tsv",sep='\t')
annots

In [ ]:
convt = annots[['Original Name', 'Species Name']]
convt

In [ ]:
convt.index = convt['Original Name'].astype(str)
convt

In [ ]:
refvals = pd.DataFrame(ref.values, index = ref.index, columns=["nmol_fraction_LCMS"])
refvals.index = refvals.index.str.replace('Hex1Cer', 'HexCer')
refvals

In [ ]:
tmp = pd.read_csv("manuallyannotated_addlcms.csv", index_col=0).dropna()
tmp

In [ ]:
refvalstmp = refvals.loc[refvals.index.isin(tmp.iloc[:,0]),:]
refvalstmp

In [ ]:
import numpy as np

rvl = np.array(refvals.index)
convl = np.array(convt.index)

len(np.intersect1d(rvl, convl)) ###### only a fraction... differences in nomenclature??

In [ ]:
np.intersect1d(rvl, convl)

In [ ]:
len(rvl)

In [ ]:
np.setdiff1d(rvl, convl) ## these are curently lost: mostly the AND/OR, plus some species

In [ ]:
pd.DataFrame(np.setdiff1d(rvl, convl)).to_csv("tomanuallyannotate.csv")

In [ ]:
annots.index = annots['Original Name']
annots = annots.loc[np.intersect1d(rvl, convl),:]
annots

In [ ]:
refvals = refvals.loc[np.intersect1d(rvl, convl),:]
refvals

In [ ]:
indivannots = annots[['Species Name']]

In [ ]:
len(indivannots.index.unique())

In [ ]:
indivannots = indivannots.groupby('Original Name').first()

In [ ]:
refvals['Species Name'] = refvals.index.map(indivannots['Species Name'])

In [ ]:
refvals

In [ ]:
tmp.index = tmp.iloc[:,0]
tmp = tmp.loc[refvalstmp.index,:]
refvalstmp['Species Name'] = tmp['Unnamed: 2']
refvalstmp

In [ ]:
quantlcms = pd.concat([refvals, refvalstmp], axis=0)
quantlcms.index = quantlcms['Species Name']
quantlcms = quantlcms[['nmol_fraction_LCMS']]
quantlcms

In [ ]:
quantlcms.to_csv("quantlcms_totals_averagemales_updatednomenclature.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.hist(quantlcms['nmol_fraction_LCMS'], bins=100)
plt.show()

In [ ]:
measuredlips = pd.read_hdf("datavignettes20240815.h5ad", key="table")

In [ ]:
measuredlips = measuredlips.iloc[:,:548]
measuredlips                          

In [ ]:
measlips = np.array(measuredlips.columns)
measlips = [entry for entry in measlips if "TIE" not in entry and "BAD" not in entry and "ISOTIE" not in entry] # start easy with those supposed not to have ties
measlips = [entry.replace("_1", "").replace("_2", "").replace("_3", "").replace("_4", "").replace("_dup", "") for entry in measlips]
measlips = [entry for entry in measlips if '.' not in entry]

len(np.unique(measlips)) # unique lipids with a name

In [ ]:
measlips

In [ ]:
len(np.intersect1d(measlips, np.array(quantlcms.index)))

In [ ]:
np.intersect1d(measlips, np.array(quantlcms.index))

In [ ]:
# what are we missing?

np.setdiff1d(measlips, np.array(quantlcms.index)) # they make sense: some escaped ties, some t --> now O2, some PA that are not in the QUANT, ...

In [ ]:
colsss = measuredlips.columns
colsss = ["CUT" if '.' in entry else entry for entry in colsss]
colsss

In [ ]:
colsss = ["CUT" if "TIE" in xxx else xxx for xxx in colsss]
colsss = ["CUT" if "BAD" in xxx else xxx for xxx in colsss]
colsss = ["CUT" if "ISOTIE" in xxx else xxx for xxx in colsss]
colsss = [entry.replace("_1", "").replace("_2", "").replace("_3", "").replace("_4", "").replace("_dup", "") for entry in colsss]
colsss

In [ ]:
measuredlips.columns = colsss
measuredlips

In [ ]:
toquant = measuredlips

In [ ]:
toquant = toquant.drop("CUT", axis=1)

In [ ]:
toquant = toquant.loc[:, np.intersect1d(measlips, np.array(quantlcms.index))]
toquant # note there are several duplicates and there might be outliers floating around

In [ ]:
totsig = pd.DataFrame(toquant.values.sum(axis=0),index=toquant.columns, columns=['totsig'])

In [ ]:
quantlcms['Species Name'] = quantlcms.index
quantlcms

In [ ]:
mean_nmols_undiscriminablespecies = quantlcms.loc[np.intersect1d(totsig.index, quantlcms.index),'nmol_fraction_LCMS'].groupby('Species Name').mean()
mean_nmols_undiscriminablespecies

In [ ]:
totsig

In [ ]:
mean_nmols_undiscriminablespecies = pd.DataFrame(mean_nmols_undiscriminablespecies)
mean_nmols_undiscriminablespecies 

In [ ]:
mean_nmols_undiscriminablespecies

In [ ]:
totsig['MeanNmols'] = totsig.index.map(mean_nmols_undiscriminablespecies.squeeze()) * 1000000000
totsig['ScaleFactor'] = totsig['MeanNmols'] / totsig['totsig']
totsig

In [ ]:
plt.scatter(totsig['totsig'][totsig['totsig']<0.1*1e6], totsig['MeanNmols'][totsig['totsig']<0.1*1e6], s=0.5, alpha=0.5)
plt.show()

In [ ]:
filtered_data = totsig[totsig['totsig'] < 0.1 * 1e6]

correlation = np.corrcoef(filtered_data['totsig'], filtered_data['MeanNmols'])[0, 1]
correlation

In [ ]:
filtered_data = totsig[totsig['totsig'] < 5000]

correlation = np.corrcoef(filtered_data['totsig'], filtered_data['MeanNmols'])[0, 1]
correlation

In [ ]:
scalef = np.array(totsig['ScaleFactor'])

In [ ]:
toquant

In [ ]:
augmented = toquant * scalef
augmented

In [ ]:
augmented.values.sum(axis=0) # great!

In [ ]:
transposed_df = augmented.transpose()

mean_df = transposed_df.groupby(transposed_df.index).mean()

averaged_df = mean_df.transpose()
averaged_df

In [ ]:
measuredlips = pd.read_hdf("datavignettes20240815.h5ad", key="table")

In [ ]:
averaged_df = pd.concat([averaged_df, measuredlips.iloc[:,548:]], axis=1)
averaged_df

In [ ]:
averaged_df.to_hdf("20240821_augmented_quantlcms_AVERAGED.h5ad", key="table")

In [ ]:
def rename_duplicated_columns(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        dup_count = 1
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(len(cols[cols == dup]))]
    df.columns = cols
    return df

augmented = rename_duplicated_columns(augmented)

In [ ]:
augmented.to_hdf("20240821_augmented_quantlcms.h5ad", key="table")

In [ ]:
augmented

In [ ]:
measuredlips[['Section', 'xccf', 'yccf', 'zccf']].to_hdf("coordinates.h5ad", key="table")